In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd

# Input-output y serialización

Una necesidad común es guardar un resultado de nuestro análisis en disco para uso posterior o para su distribución

**Serialización:** Se refiere al proceso de convertir una estructura de datos en un formato que permita su almacenamiento en el disco y su subsecuente recuperación sin pérdida

## Texto plano con `repr`

La palabra clave [`repr`](https://docs.python.org/3/library/functions.html#repr) nos permite convertir un objeto a string de forma no-ambigua

Luego podemos escribir nuestro objeto en un archivo de texto

Finalmente usamos [`eval`](https://docs.python.org/3/library/functions.html#eval) para devolverlo a objeto

In [ ]:
amazing_data = {'nombre': 'pablo', 'apellido': 'huijse', 'edad': 33, 'altura': 1.83}

with open("amazing_data", "w") as f:
    f.write(repr(amazing_data))
    
with open('amazing_data', 'r') as f:
    a = eval(f.read())

display(amazing_data, a, a['nombre'])

Podemos escribir la función `repr` para nuestras clases

In [ ]:
class Fruta:
    def __init__(self, nombre, color):
        self.nombre = nombre
        self.color = color
        
    def __repr__(self):
        return("{0}('{1}', '{2}')".format(self.__class__.__name__, self.nombre, self.color))
        
amazing_fruta = Fruta("Mango", "Amarillo")

with open("amazing_data", "w") as f:
    f.write(repr(amazing_fruta))
    
with open('amazing_data', 'r') as f:
    b = eval(f.read())
    
display(b, b.nombre, b.color)

## Texto plano: Comma-separated value (CSV)

Python provee un modelo csv que nos permite leer y escribir CSVs

In [ ]:
import csv

amazing_data = [["#","Apellido","Nombre","Nota"],[1,"Huijse","Pablo",4.0],[2,"Fulano","Detal",7.0]]

with open('amazing_data', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(amazing_data)

!cat amazing_data.csv

with open('amazing_data', newline='') as f:
    reader = csv.reader(f)
    rows = [fields for fields in reader]
    
display(rows)

Podemos manipular CSVs con NumPy usando las funciones [`loadtxt`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.loadtxt.html#numpy.loadtxt), [`genfromtxt`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.genfromtxt.html) y [`savetxt`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.savetxt.html)

Ojo: Sólo para datos númericos

In [ ]:
data = np.genfromtxt("amazing_data", delimiter=",")
display(data)
np.savetxt("amazing_data", data, delimiter=',')
!cat amazing_data

Y por supuesto también con pandas usando el atributo `to_csv` y la función `read_csv`

In [ ]:
df = pd.DataFrame(amazing_data[1:], columns=amazing_data[0])
display(df)
df.to_csv("amazing_data")
!cat amazing_data

## Texto plano: [JSON](https://en.wikipedia.org/wiki/JSON) (JavaScript Object Notation)

Originalmente para javascript hoy en día es un formato independiente

Principalmente usado para guardar tablas de tipo atributo-valor

Python provee el módulo [`json`](https://docs.python.org/3.7/library/json.html) que simplifica guardar estructuras anidadas

Clases, ndarrays y otros objetos curiosos no son automaticamente jsonificables.

- El ndarray se puede convertir a lista con el atributo `tolist()`
- [JSON-tricks](https://json-tricks.readthedocs.io/en/latest/)


In [ ]:
import json

amazing_numpy = np.random.randn(40,40)
display(amazing_numpy[:4, :4])
#amazing_numpy_jsonificado = amazing_numpy.ravel().tolist()
amazing_numpy_jsonificado =  pd.DataFrame(amazing_numpy).to_json()
amazing_data = {'d1':{'nombre': 'pablo', 'apellido': 'huijse', 'edad': 33, 'altura': 1.83}, 'd2': (1, 'gong'), 'd3': amazing_numpy_jsonificado}

with open('amazing_data', 'w', encoding='utf-8') as f:
    json.dump(amazing_data, f, sort_keys=True)
    
#!head  amazing_data

with open('amazing_data', 'r', encoding='utf-8') as f:
    a = json.load(f)
b = pd.read_json(a['d3']).values    
display(b[:4, :4])

## Tipos binarios: NumPy

Podemos guardar un ndarray con las funciones [`save`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.save.html#numpy.save) y [`load`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.load.html#numpy.load). Estas funciones son plataforma-independientes

También podemos binarizar un arreglo con el atributo `tobytes()` y luego leerlo con la función `frombuffer`. Estos no son plataforma-independientes

In [ ]:
fine_data = np.random.rand(40, 40)
display(fine_data[:2, :2])
display(fine_data.tobytes()[:10])
with open("mybinario", "wb") as f:
    np.save(f, fine_data)
with open("mybinario", "rb") as f:
    a = np.load(f)
display(a[:2, :2])

## Tipos binarios: [Pickle](https://docs.python.org/3/library/pickle.html)

Puede serializar casi cualquier objeto Python arbitrario 

Protocolo específico para Python

**Ojo:** Nunca cargues un pickle de procedencia desconocida, podría ser malicioso

Para archivos grandes `pickle` puede resultar un poco lento. En esos casos usa `_pickle` (ex `cPickle`)

[dill](https://pypi.org/project/dill/): Extensión que incrementa la cantidad de tipos que se pueden conservar

In [ ]:
import pickle
#import _pickle as pickle # Antiguo cPickle

amazing_data = {'d1':{'nombre': 'pablo', 'apellido': 'huijse', 'edad': 33, 'altura': 1.83}, 'd2': (1, 'gong'), 'd3': np.random.randn(50, 50)}

pickle.dump(amazing_data, open("mybinario", "wb"), protocol=2) # procolo 2 lo hace compatible con Python 2

a = pickle.load(open("mybinario", "rb"))
display(a)

### Mayor compresión: Pickle (u otro formato) + gzip/bz2

In [ ]:
import gzip, bz2

with gzip.open('mybinario.pgz','wb') as f:
    pickle.dump(amazing_data, f, protocol=2)
    
with bz2.open('mybinario.pbz2','wb') as f:
    pickle.dump(amazing_data, f, protocol=2)

## Grandes volumenes de datos numéricos: Hierarchical Data Format v5 (HDF5)

Formato diseñado para soportar grandes datasets y con alto rendimiento de I/O (paralelo)

Se guarda una estructura jerarquica (tipo filesystem) dentro de una archivo

Existen dos librerias principales en Python para leer HDF5: [h5py](https://www.h5py.org/) y [PyTables](https://www.pytables.org/) 

PyTables es usado por pandas y tiene un nivel de abstracción un poco más alto (modelo tipo base de datos)

Pandas tiene clases 

### Ejemplo de uso de h5py

Creamos grupos jerarquizados y guardamos nuestros datos 

Podemos guardar metadata usando atributos de grupo o de dato: `attrs`

Si queremos recuperar los objetos numpy usamos `[()]` sobre la referencia 


In [ ]:
import h5py

with h5py.File("amazing.h5", mode="w") as f: # modos disponibles r, r+, w, w-, a
    grp1 = f.create_group("amazing_data")
    grp11 = f.create_group("amazing_data/amazing_matrix")
    f["amazing_data/amazing_matrix/matrix1"] = np.arange(25).reshape(5, 5)
    f["amazing_data/amazing_matrix/matrix1"].attrs["descripción"] = "Estos datos son de perros y gatos"
    f["amazing_data/amazing_matrix/matrix1"].attrs["fecha"]  = "01/01/99"
    f["amazing_data/amazing_matrix/matrix2"] = np.random.rand(5, 5)
    f.create_dataset("amazing_data/amazing_matrix/matrix3", shape=(5, 5), fillvalue=33)
    f["amazing_data/amazing_string"] = "hola mundo"
    
    
with h5py.File("amazing.h5", mode="r") as f:
    display(f["amazing_data/amazing_matrix/matrix1"].attrs.keys(),
            f["amazing_data/amazing_matrix/matrix1"].attrs["descripción"])
    matrix1 = f["amazing_data/amazing_matrix/matrix1"][()] # Esto recupera los datos
    matrix2 = f["amazing_data/amazing_matrix/matrix2"] # Esto recupera una referencia
    matrix3 = f["amazing_data/amazing_matrix/matrix3"][()]
    display(f["amazing_data/amazing_string"], matrix2)

# Una vez que cerré el archivo la referencia queda invalida
display(np.allclose(np.arange(25).reshape(5, 5), matrix1), matrix2, matrix3)

## Extras

- [marshmellow](https://marshmallow.readthedocs.io/en/3.0/)
- JSON + binario = [msgpack](https://msgpack.org/)
- JSON + pickle = [jsonpickle](https://github.com/jsonpickle/jsonpickle)
- Google [protobuf](https://developers.google.com/protocol-buffers/)